# SK: Models, Prompts and Output Parsers


## Outline

 * Direct API calls to OpenAI
 * API calls through SK:
   * Prompts
   * Models
   * Output parsers

## Get your [OpenAI API Key](https://platform.openai.com/account/api-keys)

In [1]:
#!pip install python-dotenv
#!pip install openai

In [26]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

## Chat API : OpenAI

Let's start with a direct API calls to OpenAI.

In [27]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]


In [28]:
get_completion("What is 1+1?")

'As an AI language model, I can tell you that the answer to 1+1 is 2.'

In [29]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [30]:
style = """American English \
in a calm and respectful tone
"""

In [31]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [32]:
response = get_completion(prompt)

In [33]:
response

'I am quite upset that my blender lid came off and caused my smoothie to splatter all over my kitchen walls. Additionally, the warranty does not cover the cost of cleaning up the mess. Would you be able to assist me, please? Thank you kindly.'

## Chat API : SK

Let's try how we can do the same using Semantic Kernel.

In [34]:
#!pip install --upgrade semantic-kernel

### Model

In [35]:
import semantic_kernel as sk
import os
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

kernel=sk.Kernel()

In [36]:
api_key = os.environ['OPENAI_API_KEY']

In [37]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
#chat = ChatOpenAI(temperature=0.0)
kernel.add_chat_service(
        "chat-gpt", OpenAIChatCompletion("gpt-3.5-turbo", api_key)
    )

### Prompt template

In [38]:
template_string = """Translate the following text 
into a style that is {{ $style }}. 
text: {{ $text }}
"""

In [39]:
customer_style = """American English 
in a calm and respectful tone
"""

In [40]:
customer_email = """
Arrr, I be fuming that me blender lid 
flew off and splattered me kitchen walls 
with smoothie! And to make matters worse, 
the warranty don't cover the cost of 
cleaning up me kitchen. I need yer help 
right now, matey!
"""

In [41]:
# Call the LLM to translate to the style of the customer message

# Register a semantic function/skill based on the prompt
rewrite = kernel.create_semantic_function(template_string, temperature=0.0)
context_variables = sk.ContextVariables(variables={
    "style": customer_style,
    "text": customer_email
})
customer_response = rewrite(variables=context_variables)

In [42]:
print(customer_response)

I'm quite upset that my blender lid flew off and made a mess of my kitchen walls with smoothie. To add to my frustration, the warranty doesn't cover the cost of cleaning up my kitchen. I would greatly appreciate your assistance at this time, my friend.


In [43]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [44]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [45]:
# Pass parameters to the SK skill using context
context_variables["style"] = service_style_pirate
context_variables["text"] = service_reply

customer_response = rewrite(variables=context_variables)
print(customer_response)


Ahoy there, me hearty customer! I regret to inform ye that the warranty be not coverin' the expenses for cleanin' yer galley, as ye be at fault fer misusin' yer blender by forgettin' to put the lid on afore startin' it. 'Tis a tough break, but such be the way of the sea! Fare thee well!


## Output Parsers

Let's start with defining how we would like the LLM output to look like:

In [46]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [47]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {{ $text }}
"""

In [48]:
extractor = kernel.create_semantic_function(review_template, temperature=0.0)


In [49]:
context_variables = sk.ContextVariables(variables={
    "text": customer_review
})
response=extractor(variables=context_variables)
print(response)


{
    "gift": true,
    "delivery_days": 2,
    "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}


In [50]:
print(response.result)

{
    "gift": true,
    "delivery_days": 2,
    "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}


In [51]:
# You will get an error by running this line of code 
# because'gift' is not a dictionary
# 'gift' is a string
type(response.result)

str

### Parse the LLM output string into a Python dictionary

In [52]:
# Here we use Langchain support functions to parse the LLM Output
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [53]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [54]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [55]:
output_dict = output_parser.parse(response.result)

In [56]:
output_dict

{'gift': True,
 'delivery_days': 2,
 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}

In [57]:
type(output_dict)

dict

In [58]:
output_dict.get('delivery_days')

2